# Applied  ML and Advanced AI systems - AIDI 2000
## Final Project - Real-time mask detection using tensorflow object detection API
## Course Facilitator:

Noopa Jagdeesh

## Group Members: 

Robin Manchanda (100811316)

Krishnamohan Pingali (100820336)

Nishant Balasubramanian (100808971)

## Importing Libraries

In [6]:
import cv2 
import numpy as np
import os
import shutil
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

## Defining Path

In [7]:
WORKSPACE_PATH = 'Tensorflow/workspace'# WORKING DIR
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'# CLONED FROM Tensorflow Object Detection API
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'# To store labelmap.pbtx, train.records and test.records
IMAGE_PATH = WORKSPACE_PATH+'/images'# To store images
MODEL_PATH = WORKSPACE_PATH+'/models'# To store the custom model
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'# To store the pre trained model
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'# the config file of custom model
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'# checkpoints of custom model

## Creating label_map.pbtxt

In [8]:
# List of dictionaries with keys as name of label, and id of the label.
labels = [{'name':'Mask', 'id':1}, {'name':'No_Mask', 'id':2}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## Tensorflow Records File

To Convert all train and test .xml files to .record files, we will execute "generate_tfrecord.py" script


In [4]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


## Downloading pre-trained model (ssd_mobilenet_v2_fpnlite_320x320_coco17)

For the purpose of this project, we are not building the detection model from scratch instead we are going to levearage the transfer learning concept.

For this project, we are using ssd mobilenet v2 fpn 320*320, as this model provides good detection speed for our real time detection and also provides good mean accuracy.

Available at : http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

In [5]:
# changing directory to store pre-trained-models
!cd C:\Users\14372\Desktop\College-2\ML\Project\Tensorflow\workspace\pre-trained-models

In [6]:
!pip install wget 
import wget # wget is used to download the pretrained model

In [7]:
# Downloading pre-trained model (ssd_mobilenet_v2_fpnlite_320x320_coco17)
wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
!move ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

100% [.......................................................................] 20515344 / 20515344        1 file(s) moved.


x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


In [8]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.


In [9]:
shutil.move(PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', MODEL_PATH+'/'+CUSTOM_MODEL_NAME)

'Tensorflow/workspace/models/my_ssd_mobnet\\pipeline.config'

In [10]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [11]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [12]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
   

Now we need to make some changes in the config file.
1. num_classes
2. batch_size
3. fine_tune_checkpoint
4. fine_tune_checkpoint_type
5. label_map_path
6. Train & Test record path

In [13]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [14]:
pipeline_config.model.ssd.num_classes = 2 # as the number of classes here are two(mask, no mask)
pipeline_config.train_config.batch_size = 4 # for effective training
# As we are not training from scratch, providing check point of pretrained model
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.pipeline_config.train_config.fine_tune_checkpoint_type = "detection" # type
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt' # label map for train
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record'] # tensorflow train records
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt' # label map for train
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record'] # tensorflow train records

In [15]:
pipeline_config

model {
  ssd {
    num_classes: 2
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.01
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.997
          scale: true
          epsilon: 0.001
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      arg

The changes we have made are not stored anywhere so we will use below code to store these changes.

In [16]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

The below code is used to train the custom tensorflow object detection model, as recommended on the official website, we use anaconda prompt to train our model

In [17]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=3000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=3000


## Loading the trained model

In [9]:
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH) # get the config file
detection_model = model_builder.build(model_config=configs['model'], is_training=False) # bulding model

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model) # restoring checkpoint
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-2')).expect_partial()

@tf.function
def detect_fn(image):
    """This function is used to predict based on the image given"""
    image, shapes = detection_model.preprocess(image) # preprocessing like input shape, rescaling etc.
    prediction_dict = detection_model.predict(image, shapes) # prediction
    detections = detection_model.postprocess(prediction_dict, shapes) # postprocessing
    return detections

## Real time detection

In [10]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [12]:
category_index

{1: {'id': 1, 'name': 'Mask'}, 2: {'id': 2, 'name': 'No_Mask'}}

In [11]:
# Setup capture
cap = cv2.VideoCapture(0) # Allows to access webcam
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # width of the frame
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # height of the frame

In [5]:
while True: 
    
    ret, frame = cap.read() # reading frame
    image_np = np.array(frame) # converting to numpy array
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)# converting to tensor
    detections = detect_fn(input_tensor) # calling detect_fn
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()

## References
1. TensorFlow 2 Object Detection API tutorial, retrieved from https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/
2. Pre-trained model downloaded from https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
3. Nicholas Renotte, https://www.youtube.com/watch?v=yqkISICHH-U&t=17274s